In [14]:
#!/usr/bin/env python
# coding: utf-8
import argparse
import os
import sys
import jsonlines
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from tqdm import tqdm
import numpy as np

In [15]:
path_alpaca_lora = '/data/ydh/nlp/Self_Reflection_Medical/packages/alpaca-lora'
sys.path.append(path_alpaca_lora)
from utils.callbacks import Iteratorize, Stream
from utils.prompter import Prompter

In [5]:
path_CTRLEval = '/data/ydh/nlp/Self_Reflection_Medical/packages/CTRLEval'
sys.path.append(path_CTRLEval)
from ctrleval import CTRLEval

In [6]:
path_GPTScore = '/data/ydh/nlp/Self_Reflection_Medical/packages/GPTScore'
sys.path.append(path_GPTScore)
from gpt3_score import gpt3score

In [13]:
path_self = '/data/ydh/nlp/Self_Reflection_Medical'
sys.path.append(path_self)
from evaluate.sent_similarity import Sent_Similar
from evaluate.loop_eval_utils import evaluate_response, evaluate_knowledge
from loop_utils import main_loop

loop test

In [8]:
import json
import jsonlines
from tqdm import tqdm
import copy
input_file = '/data/ydh/nlp/dataset/test_one.jsonl'
out_file = '/data/ydh/nlp/test_out.jsonl'
testcase = 'exp1'
# 항상 처음부터 실행
with jsonlines.open(input_file) as reader:
    reader = list(reader)
    for i, line in tqdm(enumerate(reader), total=len(reader)):
        line_copy = copy.deepcopy(line)
        candidate_justifications = {}
        if testcase == 'exp1':  # Self-Reflection
            for candidate, data in line_copy['candidates'].items():
                temp_line = {
                    'id': line['id'],
                    'question': line['question'],
                    'candidate': candidate,
                    'is_correct': data['predicted']
                }
                u_loop = False
                final_knowledge, final_response, all_history_knowledge, all_history_response = 1,2,3,4
                candidate_justifications[candidate] = {             
                        'justification': final_response,
                        'history_knowledge': all_history_knowledge,
                        'history_response': all_history_response,
                        'generated_knowledge': final_knowledge
                    }
            line_copy["candidate_answers"] = candidate_justifications
            del line_copy['candidates']  # Remove candidates from the output
            # Write results
            writer = jsonlines.open(out_file, mode='a')
            print(line_copy)
            writer.write(line_copy)
            writer.close()
        elif testcase == 'exp2':  # Classifier + Slef-Reflextion
            for candidate_text, pred in line['candidates'].items():
                temp_line = {
                    'id': line['id'],
                    'question': line['question'],
                    'candidate': candidate_text,
                    'is_correct': pred['predicted']
                }
                final_knowledge, final_response, all_history_knowledge, all_history_response = 1,2,3,4
                candidate_justifications[candidate_text] = {             
                            'justification': final_response,
                            'history_knowledge': all_history_knowledge,
                            'history_response': all_history_response,
                            'generated_knowledge': final_knowledge
                        }
            line["candidate_answers"] = candidate_justifications
            # Write results
            writer = jsonlines.open(out_file, mode='a')
            print(line)
            writer.write(line)
            writer.close()


100%|██████████| 1/1 [00:00<00:00, 48.71it/s]

{'id': 'trivia_254', 'question': 'Which crusade was led by Philip II Augustus of France and Richard I of England?', 'candidate_answers': {'2nd Crusade': {'justification': 2, 'history_knowledge': 3, 'history_response': 4, 'generated_knowledge': 1}, '4th Crusade': {'justification': 2, 'history_knowledge': 3, 'history_response': 4, 'generated_knowledge': 1}, '5th Crusade': {'justification': 2, 'history_knowledge': 3, 'history_response': 4, 'generated_knowledge': 1}, '3rd Crusade': {'justification': 2, 'history_knowledge': 3, 'history_response': 4, 'generated_knowledge': 1}}}
